<a href="https://colab.research.google.com/github/greyhound101/multilingial/blob/master/classifier_data_es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [2]:
pip install transformers

     |████████████████████████████████| 1.4MB 9.6MB/s 
     |████████████████████████████████| 2.9MB 20.5MB/s 
     |████████████████████████████████| 890kB 27.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=5cfa0fbde971db11f4658ce2d17c3dc01738e423ff1a837005fb87792b4d5b81
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
pip install git+https://github.com/n-waves/multifit

  Cloning https://github.com/n-waves/multifit to /tmp/pip-req-build-14psu_7v
  Running command git clone -q https://github.com/n-waves/multifit /tmp/pip-req-build-14psu_7v
  Created wheel for multifit: filename=multifit-1.0-cp36-none-any.whl size=24505 sha256=85d879ae7de81a0a255d6e7b7aa2168f501d461ed754b7392c58787638e789dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-prlghcdk/wheels/b7/92/1e/246f31a4e84fd665b5907cb96765f696be45e814ff68a5fd4a
Successfully built multifit


In [4]:
pip install sacremoses

In [5]:
pip install sentencepiece

     |████████████████████████████████| 1.1MB 8.8MB/s 


In [8]:
import glob
glob.glob('/content/gdrive/My Drive/multilingual/*')

['/content/gdrive/My Drive/multilingual/jigsaw-toxic-comment-train.csv',
 '/content/gdrive/My Drive/multilingual/jigsaw-toxic-comment-train-processed-seqlen128.csv',
 '/content/gdrive/My Drive/multilingual/validation.csv',
 '/content/gdrive/My Drive/multilingual/test.csv',
 '/content/gdrive/My Drive/multilingual/submission.csv',
 '/content/gdrive/My Drive/multilingual/es']

In [9]:
import pandas as pd
def load_data():
    trn=pd.read_csv('/content/gdrive/My Drive/multilingual/jigsaw-toxic-comment-train.csv',usecols=['toxic','comment_text'])
    tst=pd.read_csv('/content/gdrive/My Drive/multilingual/test.csv',usecols=['lang','content'])  
    sub=pd.read_csv('/content/gdrive/My Drive/multilingual/submission.csv')  
    return trn,tst,sub
train,test,sub=load_data()
test['toxic']=sub['toxic'].round()

In [10]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tensorflow.keras.layers import *
from fastai.text import *
import multifit

In [11]:
es=test.loc[test['lang']=='es'].reset_index(drop=True)
es.head()

,content,lang,toxic
0,el skate es unos de los deportes favoritos de ...,es,0.0
1,Me doy la bienvenida. A este usuari le gusta c...,es,0.0
2,"ES NOTABLEMENTE TENDENCIOSO, NO SE HABLA DE CU...",es,0.0
3,"El Jardín de infantes Nº938, fundado en 1989, ...",es,0.0
4,Daré explicaciones y/o aclaraciones a cualquie...,es,0.0


In [12]:
def set_seed(seed):
    if seed is not None:
        torch.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        np.random.seed(seed)

In [13]:
exp = multifit.from_pretrained("es_multifit_paper_version")
tokenizer = exp.pretrain_lm.tokenizer
fa_config =  tokenizer.get_fastai_config(add_open_file_processor=True)
data_lm = (TextList.from_df(es, **fa_config)
            .split_by_rand_pct(0.1)
            .label_for_lm()           
            .databunch(bs=32))
exp.finetune_lm.arch.lang = 'es'
set_seed(42)
learn = exp.finetune_lm.get_learner(data_lm)  
    # learn is a preconfigured fastai learner with a pretrained model loaded
data_lm.show_batch()

Training args:  {'drop_mult': 0.3, 'true_wd': False, 'wd': 1e-07, 'pretrained': False, 'clip': 0.12} config:  {'emb_sz': 400, 'n_hid': 1550, 'n_layers': 4, 'pad_token': 1, 'qrnn': True, 'bidir': False, 'output_p': 0.25, 'hidden_p': 0.1, 'input_p': 0.2, 'embed_p': 0.02, 'weight_p': 0.15, 'tie_weights': True, 'out_bias': True}
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Setting LM training seed seed to 0
Loading pretrained weights:  [PosixPath('/root/.fastai/models/es_multifit_paper_version/lm_best'), PosixPath('/root/.fastai/models/es_multifit_paper_version/itos')]


/usr/local/lib/python3.6/dist-packages/fastai/text/data.py:339: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  idx_min = (t != self.pad_idx).nonzero().min()


idx,text
0,"us ua ri ▁le ▁gu sta ▁confronta rse ▁con ▁los ▁ignora ntes ▁. ▁ ¿ ▁ xxmaj ▁va mos ▁a ▁estar ▁ tú ▁y ▁y o ▁de ▁acuerdo ▁o ▁no ▁ ? ▁( ▁discusión ▁) ▁18 ▁: ▁28 ▁27 ▁ dic ▁2014 ▁( ▁ xxup ▁ ut c ▁) ▁ xxbos ▁ xxfld ▁1 ▁ xxmaj ▁el ▁ xxmaj ▁jardín ▁de ▁infante s ▁ xxmaj ▁no 9 38 ▁, ▁fundado"
1,"▁su ▁ xxup ▁pu ▁ xxup ▁ ta ▁ xxup ▁madre ▁, ▁ xxup ▁ asi ▁ xxup ▁es ▁, ▁ xxup ▁lo ▁ xxup ▁ vo y ▁a ▁ xxup ▁matar ▁a ▁ xxup ▁ us ted ▁ xxup ▁ asi ▁ xxup ▁que ▁ xxup ▁mejor ▁ xxup ▁viva ▁ xxup ▁bien ▁ xxup ▁hasta ▁ xxup ▁entonces ▁ xxbos ▁ xxfld ▁1 ▁ xxmaj ▁no ▁pasa ▁nada"
2,"xxmaj ▁en ▁lo ▁personal ▁, ▁no ▁ ve o ▁por ▁qué ▁he ▁pro ce di do ▁ inde bi da mente ▁; ▁en ▁fin ▁, ▁se ▁le ▁ da rá ▁el ▁beneficio ▁de ▁la ▁duda ▁y ▁tal ve z ▁en ▁esta ▁ocasión ▁de cida ▁dar ▁su ▁versión ▁y ▁solicita r ▁un ▁posible ▁de s b lo que o ▁... ▁en ▁ese ▁caso ▁, ▁que ▁otro ▁biblioteca rio ▁lo ▁e val"
3,"▁así ▁que ▁una ▁le gu mino sa ▁ahora ▁es ▁un ▁hi men ó pter o ▁, ▁también ▁ ? ▁y ▁... ▁ ¿ ▁ xxmaj ▁qué ▁co jo nes ▁ – ▁con ▁o ▁sin ▁, ▁no ▁sé ▁per dón - ▁es ▁la ▁ u ▁a cent u ada ▁y ▁al ▁modo ▁francés ▁ ? ▁ xxup ▁bar ▁ u ▁ xxup ▁ lea ▁ → ▁ xxmaj ▁ ap ó"
4,"▁de ▁que ▁sabe s ▁bastante ▁acerca ▁de ▁esta ▁banda ▁y ▁ qui zas ▁ tus ▁argumento s ▁sean ▁mas ▁que ▁va idos ▁para ▁evitar ▁que ▁ese ▁arti culo ▁se ▁bo rre . ▁esp ero ▁tu ▁respuesta ▁. ▁ xxmaj ▁un ▁salud o ▁ xxbos ▁ xxfld ▁1 ▁ xxmaj ▁pues ▁sí ▁, ▁para ▁ mí ▁también ▁... ▁esto y ▁desea ndo ▁escuchar ▁la ▁entrevista ▁de ▁la ▁radio ▁, ▁con ▁eso"


In [14]:
# learn.freeze_to(-1)
# learn.fit_one_cycle(1, 1e-4 * 10, moms=(0.8, 0.7))
# learn.unfreeze()
# learn.fit_one_cycle(10, 1e-4, moms=(0.8, 0.7))

In [15]:
path='/content/gdrive/My Drive/multilingual toxic/es/es1'
self=exp.finetune_lm
CLS_BEST = path+'/cls_best1'
LM_BEST = path+"/lm_best1"
ENC_BEST = path+"/enc_best1"

In [16]:
path='/content/gdrive/My Drive/multilingual toxic/es/es1'
self=exp.finetune_lm
CLS_BEST = path+'/cls_best'
LM_BEST = path+"/lm_best"
ENC_BEST = path+"/enc_best"


experiment_path=Path(path)
self.experiment_path=experiment_path
print("Experiment", experiment_path)

#save tokenizer
tokenizer.save(self.experiment_path, learn=learn)
learn.to_fp32()
#save encoder weights to (ENC_BEST,learn.path,learn.model_dir)
learn.save_encoder(ENC_BEST)
#save model structure to (LM_BEST, learn.path,learn.model_dir,'.pth')
learn.save(LM_BEST, with_opt=False)
# learn.destroy()
# saving 'seed', 'name', 'arch', 'experiment_path', 'dataset_path', 'num_epochs', 'bs', 'bptt', 'drop_mult', 'dropout_values', 'label_smoothing_eps', 'label_smoothing_eps_norm_by_classes', 'use_adam_08', 'true_wd', 'wd', 'clip', 'fp16', 'lr', 'base' 
# to /content/gdrive/My Drive/multilingual toxic/es/es1/finetuning.json
# save_paramters(self)

Experiment /content/gdrive/My Drive/multilingual toxic/es/es1
Copy sp model from /root/.fastai/models/es_multifit_paper_version to /content/gdrive/My Drive/multilingual toxic/es/es1


In [17]:
self=exp.classifier
es['toxic']=es['toxic'].astype(np.long)
from pathlib import Path
data_clas = (TextList.from_df(es,cols=['content','toxic'], **fa_config)
            .split_by_rand_pct(0.1)
            .label_from_df('toxic')           
            .databunch(bs=32))
l1rn = exp.classifier.get_learner(data_clas)  
# learn is a preconfigured fastai learner with a pretrained model loaded
data_clas.show_batch()

Using Label smoothing with eps =  0.05
Setting Classifier weights seed seed to 0
Training args:  {'drop_mult': 0.5, 'wd': 0.01, 'pretrained': False, 'bptt': 70, 'loss_func': FlattenedLoss of LabelSmoothingCrossEntropy(), 'clip': 0.12, 'silent': False} config:  {'emb_sz': 400, 'n_hid': 1550, 'n_layers': 4, 'pad_token': 1, 'qrnn': True, 'bidir': False, 'output_p': 0.25, 'hidden_p': 0.1, 'input_p': 0.2, 'embed_p': 0.02, 'weight_p': 0.15}
Loading pretrained model /content/gdrive/My Drive/multilingual toxic/es/es1/enc_best
Setting Classifier training seed seed to 0


text,target
▁ xxbos ▁ xxfld ▁1 ▁http ▁: ▁/ ▁/ ▁www . the - af c . com ▁/ ▁en ▁/ ▁e vent - in form a tion ▁/ ▁ af c - asi an - cu p - ma tch - off icia l s ▁http ▁: ▁/ ▁/ ▁www . the - af c . com ▁/ ▁en ▁/ ▁compone nt ▁/ ▁ jo om lea gue ▁/ ▁,0
▁ xxbos ▁ xxfld ▁1 ▁ xxup ▁san ▁ xxup ▁ ba udi lio ▁ n ▁ xxrep ▁ 44 ▁o ▁ xxup ▁san ▁ xxup ▁ ba udi lio ▁ n ▁ xxrep ▁ 44 ▁o ▁ xxup ▁san ▁ xxup ▁ ba udi lio ▁ n ▁ xxrep ▁ 44 ▁o ▁ xxup ▁san ▁ xxup ▁ ba udi lio ▁ n ▁ xxrep ▁ 44 ▁o ▁,0
"▁ xxbos ▁ xxfld ▁1 ▁ xxup ▁señor ▁ xxup ▁ iva n ▁ xxup ▁ ce pe da ▁, ▁ xxup ▁duro ▁ xxup ▁ cri tico ▁ xxup ▁del ▁ xxup ▁mejor ▁ xxup ▁presidente ▁ xxup ▁que ▁ xxup ▁he mos ▁ xxup ▁tenido s ▁ xxup ▁los ▁ xxup ▁colombiano s ▁ xxup ▁ vic tima ▁ xxup ▁de ▁ xxup ▁las ▁ xxup ▁a tro",1
"▁ xxbos ▁ xxfld ▁1 ▁no ▁mas ▁de s on rra s ▁chica s ▁... ▁re be nte mos ▁una ▁re pres a ▁, ▁como ▁, ▁ ra pel ▁, ▁mau le ▁. . ▁ oh ▁haciendo la ▁ xxup ▁esta lla r ▁... ▁ xxup ▁ca os ▁ xxup ▁total ▁ xxup ▁en ▁ xxup ▁chile ▁... ▁ xxup ▁honra ▁ xxup ▁ ah ▁ xxup ▁nivel ▁ xxup",1
"▁ xxbos ▁ xxfld ▁1 ▁ xxup ▁cada ▁ xxup ▁ dia ▁ xxup ▁es ▁ xxup ▁mas ▁ xxup ▁evidente ▁ xxup ▁que ▁ xxup ▁el ▁ xxup ▁pueblo ▁ xxup ▁colombiano ▁ xxup ▁rechaza ▁ xxup ▁cada ▁ xxup ▁vez ▁ xxup ▁mas ▁ xxup ▁los ▁ xxup ▁actos ▁, ▁ xxup ▁poli tica s ▁e ▁ xxup ▁ ide ologi a ▁ xxup ▁po dri da ▁",0


In [18]:
l1rn.unfreeze()
def _fit_schedule_1cycle(num_epochs, learn):
        learn.unfreeze()
        learn.fit_one_cycle(num_epochs, slice(1e-2 / (2.6 ** 4), 2e-2), moms=(0.8, 0.7))

def _fit_schedule_layered(num_epochs, learn):
        learn.freeze_to(-1)
        learn.fit_one_cycle(1, 2e-2, moms=(0.8, 0.7))
        if num_epochs > 1:
            learn.freeze_to(-2)
            learn.fit_one_cycle(1, slice(1e-2 / (2.6 ** 4), 1e-2), moms=(0.8, 0.7))
            learn.freeze_to(-3)
            learn.fit_one_cycle(1, slice(5e-3 / (2.6 ** 4), 5e-3), moms=(0.8, 0.7))
            learn.unfreeze()
        if num_epochs > 5:
            learn.fit_one_cycle(num_epochs - 4, slice(1e-3 / (2.6 ** 4), 1e-3), moms=(0.8, 0.7))

def _fit_schedule_2cycle(num_epochs, learn):
        learn.freeze_to(-1)
        learn.fit_one_cycle(1, 2e-2, moms=(0.8, 0.7))
        learn.unfreeze()
        if num_epochs > 1:
            learn.fit_one_cycle(num_epochs - 1, slice(1e-2 / (2.6 ** 4), 1e-2), moms=(0.8, 0.7))

def _fit_schedule_reverse_2cycle(num_epochs, learn):
        learn.unfreeze()
        for g in learn.layer_groups[-1:]:
            for l in g:
                if not learn.train_bn or not isinstance(l, bn_types): requires_grad(l, False)
        learn.create_opt(defaults.lr)
        learn.fit_one_cycle(num_epochs, slice(1e-2 / (2.6 ** 4), 2e-2), moms=(0.8, 0.7))
        learn.unfreeze()
        learn.fit_one_cycle(num_epochs, slice(1e-3 / (2.6 ** 4), 2e-3), moms=(0.8, 0.7))

def _fit_schedule_false_wd(self, learn):
        learn.true_wd = False
        learn.fit_one_cycle(1, 5e-2, moms=(0.8, 0.7), wd=1e-7)
        if num_epochs > 1:
            learn.freeze_to(-2)
            learn.fit_one_cycle(1, slice(5e-2 / (2.6 ** 4), 5e-2), moms=(0.8, 0.7), wd=1e-7)
            learn.freeze_to(-3)
            learn.fit_one_cycle(1, slice(5e-4 / (2.6 ** 4), 5e-4), moms=(0.8, 0.7), wd=1e-7)
            learn.unfreeze()
            if num_epochs > 5:
                learn.fit_one_cycle(num_epochs - 4, slice(1e-2 / (2.6 ** 4), 1e-2), moms=(0.8, 0.7), wd=1e-7)

# _fit_schedule_1cycle(10,l1rn)

In [19]:
self.experiment_path = l1rn.path / l1rn.model_dir
tokenizer.save(self.experiment_path, learn=l1rn)
l1rn.to_fp32()
l1rn.save(CLS_BEST, with_opt=False)
print("Classifier model saved to", self.experiment_path)
self.save_paramters()
        # learn.destroy()

Copy sp model from /root/.fastai/models/es_multifit_paper_version to multifit_paper_version
Classifier model saved to multifit_paper_version
Saving dump to multifit_paper_version/classifier.json


'{\n  "seed": 0,\n  "name": "multifit_paper_version",\n  "arch": {\n    "tokenizer_type": "sp",\n    "max_vocab": 15000,\n    "lang": "es",\n    "emb_sz": 400,\n    "n_hid": 1550,\n    "n_layers": 4,\n    "qrnn": true\n  },\n  "experiment_path": "multifit_paper_version",\n  "dataset_path": null,\n  "bs": 18,\n  "num_epochs": 8,\n  "drop_mult": 0.5,\n  "dropout_values": {\n    "output_p": 0.25,\n    "hidden_p": 0.1,\n    "input_p": 0.2,\n    "embed_p": 0.02,\n    "weight_p": 0.15\n  },\n  "wd": 0.01,\n  "clip": 0.12,\n  "label_smoothing_eps": 0.1,\n  "label_smoothing_eps_norm_by_classes": true,\n  "weighted_cross_entropy": null,\n  "early_stopping": null,\n  "fit_schedule": "1cycle",\n  "random_init": false,\n  "bptt": 70,\n  "fp16": false,\n  "base": "/content/gdrive/My Drive/multilingual toxic/es/es1"\n}'